In [1]:
import pandas as pd
import numpy as np

In [2]:
risk_qa = pd.read_excel("risk_data.xlsx")

In [9]:
risk_qa

,客戶編號,風險等級,風險等級分類,適性評量填寫日期,適性版號,題目序號,KYC_QUESTION,ANS_ORDER,ANS_ITEM,答題得分,客戶職業與任職部門,AGENT代碼
0,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A0,是否為其他國家稅務居住者?,1,1.否,0,NaN,9999999999
1,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A01,職業／行業別,8,8.退休人士,0,NaN,9999999999
2,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A02,職務,1,1.基層職員,0,NaN,9999999999
3,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A03,任職單位,1,1.任職公司／機構,0,寶豐,9999999999
4,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A03,任職單位,2,2.任職部門,0,營建,9999999999
5,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A04,學歷,4,4.大學,0,NaN,9999999999
6,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A05,婚姻狀況,3,3.不同意提供本項資料,0,NaN,9999999999
7,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A06,外國人士居留目的,1,1.為本國人士,0,NaN,9999999999
8,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A07,與政治人物之關係,1,1.與政治人物無關,0,NaN,9999999999
9,261909,5,積極型,2018-10-30 22:54:34.523,201809A,A08,投資目的(可複選),1,1.閒置資金運用,0,NaN,9999999999


In [12]:
risk_qa.loc[risk_qa["客戶編號"]==318629,]

,客戶編號,風險等級,風險等級分類,適性評量填寫日期,適性版號,題目序號,KYC_QUESTION,ANS_ORDER,ANS_ITEM,答題得分,客戶職業與任職部門,AGENT代碼
55680,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A0,是否為其他國家稅務居住者?,1,1.否,0,NaN,A0769
55681,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A01,職業／行業別,5,5.服務業,0,NaN,A0769
55682,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A02,職務,2,2.專業人員,0,NaN,A0769
55683,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A03,任職單位,1,1.任職公司／機構,0,娛樂公司,A0769
55684,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A03,任職單位,2,2.任職部門,0,經紀人,A0769
55685,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A04,學歷,3,3.專科,0,NaN,A0769
55686,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A05,婚姻狀況,3,3.不同意提供本項資料,0,NaN,A0769
55687,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A06,外國人士居留目的,1,1.為本國人士,0,NaN,A0769
55688,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A07,與政治人物之關係,1,1.與政治人物無關,0,NaN,A0769
55689,318629,5,積極型,2019-01-14 10:23:08.907,201809A,A08,投資目的(可複選),1,1.閒置資金運用,0,NaN,A0769


In [6]:
risk_qa.pivot(index='客戶編號', columns='題目序號', values='ANS_ORDER')

ValueError: Index contains duplicate entries, cannot reshape